In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create Spark session
spark = SparkSession.builder.appName("Set3_EmployeeProject").getOrCreate()

# Employee Data
employee_data = [
    ("Ananya", "HR", 52000),
    ("Rahul", "Engineering", 65000),
    ("Priya", "Engineering", 60000),
    ("Zoya", "Marketing", 48000),
    ("Karan", "HR", 53000),
    ("Naveen", "Engineering", 70000),
    ("Fatima", "Marketing", 45000)
]
columns_emp = ["Name", "Department", "Salary"]
df_emp = spark.createDataFrame(employee_data, columns_emp)

# Performance Data
performance_data = [
    ("Ananya", 2023, 4.5),
    ("Rahul", 2023, 4.9),
    ("Priya", 2023, 4.3),
    ("Zoya", 2023, 3.8),
    ("Karan", 2023, 4.1),
    ("Naveen", 2023, 4.7),
    ("Fatima", 2023, 3.9)
]
columns_perf = ["Name", "Year", "Rating"]
df_perf = spark.createDataFrame(performance_data, columns_perf)

# Project Data
project_data = [
    ("Ananya", "HR Portal", 120),
    ("Rahul", "Data Platform", 200),
    ("Priya", "Data Platform", 180),
    ("Zoya", "Campaign Tracker", 100),
    ("Karan", "HR Portal", 130),
    ("Naveen", "ML Pipeline", 220),
    ("Fatima", "Campaign Tracker", 90)
]
columns_proj = ["Name", "Project", "HoursWorked"]
df_proj = spark.createDataFrame(project_data, columns_proj)


In [0]:
df_joined = df_emp \
    .join(df_perf, on="Name", how="inner") \
    .join(df_proj, on="Name", how="inner")

df_joined.show()


+------+-----------+------+----+------+----------------+-----------+
|  Name| Department|Salary|Year|Rating|         Project|HoursWorked|
+------+-----------+------+----+------+----------------+-----------+
|Ananya|         HR| 52000|2023|   4.5|       HR Portal|        120|
| Priya|Engineering| 60000|2023|   4.3|   Data Platform|        180|
| Rahul|Engineering| 65000|2023|   4.9|   Data Platform|        200|
|  Zoya|  Marketing| 48000|2023|   3.8|Campaign Tracker|        100|
| Karan|         HR| 53000|2023|   4.1|       HR Portal|        130|
|Naveen|Engineering| 70000|2023|   4.7|     ML Pipeline|        220|
|Fatima|  Marketing| 45000|2023|   3.9|Campaign Tracker|         90|
+------+-----------+------+----+------+----------------+-----------+



In [0]:
df_joined.groupBy("Department").agg(sum("HoursWorked").alias("TotalHours")).show()


+-----------+----------+
| Department|TotalHours|
+-----------+----------+
|         HR|       250|
|Engineering|       600|
|  Marketing|       190|
+-----------+----------+



In [0]:
df_joined.groupBy("Project").agg(avg("Rating").alias("AvgRating")).show()


+----------------+------------------+
|         Project|         AvgRating|
+----------------+------------------+
|       HR Portal|               4.3|
|   Data Platform|               4.6|
|Campaign Tracker|3.8499999999999996|
|     ML Pipeline|               4.7|
+----------------+------------------+



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Define schema matching df_perf
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Rating", DoubleType(), True)  # Explicitly say it's DoubleType
])

# Create DataFrame with a row that has a null Rating
extra_row = spark.createDataFrame([("Temp", 2023, None)], schema=schema)

# Union with original DataFrame
df_perf_null = df_perf.unionByName(extra_row)
df_perf_null.show()


+------+----+------+
|  Name|Year|Rating|
+------+----+------+
|Ananya|2023|   4.5|
| Rahul|2023|   4.9|
| Priya|2023|   4.3|
|  Zoya|2023|   3.8|
| Karan|2023|   4.1|
|Naveen|2023|   4.7|
|Fatima|2023|   3.9|
|  Temp|2023|  NULL|
+------+----+------+



In [0]:
df_perf_null.filter(col("Rating").isNull()).show()


+----+----+------+
|Name|Year|Rating|
+----+----+------+
|Temp|2023|  NULL|
+----+----+------+



In [0]:
# Join to get department for each employee
df_perf_with_dept = df_perf_null.join(df_emp, on="Name", how="left")

# Calculate avg rating per department (excluding nulls)
avg_rating_per_dept = df_perf_with_dept.groupBy("Department") \
    .agg(avg("Rating").alias("DeptAvgRating"))

# Join back to get dept average for each row
df_with_avg = df_perf_with_dept.join(avg_rating_per_dept, on="Department", how="left")

# Fill null ratings
df_filled = df_with_avg.withColumn(
    "RatingFilled", when(col("Rating").isNull(), col("DeptAvgRating")).otherwise(col("Rating"))
)
df_filled.select("Name", "Department", "Rating", "RatingFilled").show()


+------+-----------+------+------------+
|  Name| Department|Rating|RatingFilled|
+------+-----------+------+------------+
|Ananya|         HR|   4.5|         4.5|
| Rahul|Engineering|   4.9|         4.9|
| Priya|Engineering|   4.3|         4.3|
|  Zoya|  Marketing|   3.8|         3.8|
| Karan|         HR|   4.1|         4.1|
|Naveen|Engineering|   4.7|         4.7|
|Fatima|  Marketing|   3.9|         3.9|
|  Temp|       NULL|  NULL|        NULL|
+------+-----------+------+------------+



In [0]:
df_with_perf_cat = df_joined.withColumn(
    "PerformanceCategory",
    when(col("Rating") >= 4.7, "Excellent")
    .when((col("Rating") >= 4.0) & (col("Rating") < 4.7), "Good")
    .otherwise("Average")
)
df_with_perf_cat.select("Name", "Rating", "PerformanceCategory").show()


+------+------+-------------------+
|  Name|Rating|PerformanceCategory|
+------+------+-------------------+
|Ananya|   4.5|               Good|
| Priya|   4.3|               Good|
| Rahul|   4.9|          Excellent|
|  Zoya|   3.8|            Average|
| Karan|   4.1|               Good|
|Naveen|   4.7|          Excellent|
|Fatima|   3.9|            Average|
+------+------+-------------------+



In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def assign_bonus(hours):
    return 10000 if hours > 200 else 5000

bonus_udf = udf(assign_bonus, IntegerType())

df_with_bonus = df_with_perf_cat.withColumn("Bonus", bonus_udf(col("HoursWorked")))
df_with_bonus.select("Name", "HoursWorked", "Bonus").show()


+------+-----------+-----+
|  Name|HoursWorked|Bonus|
+------+-----------+-----+
|Ananya|        120| 5000|
| Rahul|        200| 5000|
| Priya|        180| 5000|
|  Zoya|        100| 5000|
| Karan|        130| 5000|
|Naveen|        220|10000|
|Fatima|         90| 5000|
+------+-----------+-----+



In [0]:
df_with_dates = df_with_bonus.withColumn("JoinDate", lit("2021-06-01").cast("date"))
df_with_dates = df_with_dates.withColumn(
    "MonthsWorked",
    floor(datediff(current_date(), col("JoinDate")) / 30)
)
df_with_dates.select("Name", "JoinDate", "MonthsWorked").show()


+------+----------+------------+
|  Name|  JoinDate|MonthsWorked|
+------+----------+------------+
|Ananya|2021-06-01|          49|
| Priya|2021-06-01|          49|
| Rahul|2021-06-01|          49|
|  Zoya|2021-06-01|          49|
| Karan|2021-06-01|          49|
|Naveen|2021-06-01|          49|
|Fatima|2021-06-01|          49|
+------+----------+------------+



In [0]:
df_with_dates.filter(col("JoinDate") < lit("2022-01-01")).select("Name", "JoinDate").show()


+------+----------+
|  Name|  JoinDate|
+------+----------+
|Ananya|2021-06-01|
| Priya|2021-06-01|
| Rahul|2021-06-01|
|  Zoya|2021-06-01|
| Karan|2021-06-01|
|Naveen|2021-06-01|
|Fatima|2021-06-01|
+------+----------+



In [0]:
extra_employees = [
    ("Meena", "HR", 48000),
    ("Raj", "Marketing", 51000)
]
df_extra = spark.createDataFrame(extra_employees, columns_emp)

df_union = df_emp.unionByName(df_extra)
df_union.show()


+------+-----------+------+
|  Name| Department|Salary|
+------+-----------+------+
|Ananya|         HR| 52000|
| Rahul|Engineering| 65000|
| Priya|Engineering| 60000|
|  Zoya|  Marketing| 48000|
| Karan|         HR| 53000|
|Naveen|Engineering| 70000|
|Fatima|  Marketing| 45000|
| Meena|         HR| 48000|
|   Raj|  Marketing| 51000|
+------+-----------+------+



In [0]:
df_read_back = spark.read.parquet("/path/to/output/final_employee_data_partitioned")
df_read_back.show()



+------+------+----+------+----------------+-----------+-------------------+-----+----------+------------+-----------+
|  Name|Salary|Year|Rating|         Project|HoursWorked|PerformanceCategory|Bonus|  JoinDate|MonthsWorked| Department|
+------+------+----+------+----------------+-----------+-------------------+-----+----------+------------+-----------+
| Rahul| 65000|2023|   4.9|   Data Platform|        200|          Excellent| 5000|2021-06-01|          49|Engineering|
| Priya| 60000|2023|   4.3|   Data Platform|        180|               Good| 5000|2021-06-01|          49|Engineering|
|Fatima| 45000|2023|   3.9|Campaign Tracker|         90|            Average| 5000|2021-06-01|          49|  Marketing|
|  Zoya| 48000|2023|   3.8|Campaign Tracker|        100|            Average| 5000|2021-06-01|          49|  Marketing|
|Naveen| 70000|2023|   4.7|     ML Pipeline|        220|          Excellent|10000|2021-06-01|          49|Engineering|
|Ananya| 52000|2023|   4.5|       HR Portal|    